<a href="https://colab.research.google.com/github/Gopi138942/Voice_translator/blob/main/voice_cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First install required packages
!pip install openai-whisper moviepy pydub googletrans==4.0.0-rc1 TTS ffmpeg-python
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!pip install --upgrade --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6

In [ ]:

import os
import whisper
from moviepy.editor import VideoFileClip, AudioFileClip
from pydub import AudioSegment
from googletrans import Translator
from TTS.api import TTS
import tempfile


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:


class SimpleVideoTranslator:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def extract_audio(self, video_path):
        """Extract audio using moviepy without temporary files"""
        with VideoFileClip(video_path) as video:
            return video.audio

    def transcribe(self, audio_clip):
        """Transcribe using Whisper directly"""
        model = whisper.load_model("base", device=self.device)

        # Save audio to temp file for Whisper
        with tempfile.NamedTemporaryFile(suffix=".wav") as tmp:
            audio_clip.write_audiofile(tmp.name, logger=None)
            result = model.transcribe(tmp.name)
        return result["text"]

    def translate_text(self, text, target_lang="es"):
        """Translate using googletrans"""
        translator = Translator()
        translation = translator.translate(text, dest=target_lang)
        return translation.text

    def synthesize_speech(self, text, reference_audio_clip):
        """Synthesize speech using TTS with voice cloning"""
        tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts").to(self.device)

        # Save reference audio to temp file
        with tempfile.NamedTemporaryFile(suffix=".wav") as ref_tmp:
            reference_audio_clip.write_audiofile(ref_tmp.name, logger=None)

            # Synthesize to temp file
            with tempfile.NamedTemporaryFile(suffix=".wav") as out_tmp:
                tts.tts_to_file(
                    text=text,
                    speaker_wav=ref_tmp.name,
                    file_path=out_tmp.name
                )
                return AudioFileClip(out_tmp.name)

    def process_video(self, input_path, output_path, target_lang="es"):
        """Complete processing pipeline"""
        try:
            # 1. Extract audio
            print("Extracting audio...")
            original_audio = self.extract_audio(input_path)

            # 2. Transcribe
            print("Transcribing audio...")
            original_text = self.transcribe(original_audio)
            print(f"Original: {original_text[:100]}...")

            # 3. Translate
            print("Translating text...")
            translated_text = self.translate_text(original_text, target_lang)
            print(f"Translated: {translated_text[:100]}...")

            # 4. Synthesize
            print("Synthesizing new audio...")
            new_audio = self.synthesize_speech(translated_text, original_audio)

            # 5. Combine with video
            print("Creating output video...")
            with VideoFileClip(input_path) as video:
                final = video.set_audio(new_audio)
                final.write_videofile(
                    output_path,
                    codec="libx264",
                    audio_codec="aac",
                    threads=4,
                    verbose=False
                )

            print(f"Success! Output saved to {output_path}")

        except Exception as e:
            print(f"Error: {str(e)}")
            raise

# Example usage
if __name__ == "__main__":
    # Download sample video
    !wget -q -O input.mp4 https://sample-videos.com/video123/mp4/720/big_buck_bunny_720p_1mb.mp4

    translator = SimpleVideoTranslator()
    translator.process_video("input.mp4", "output.mp4", "es")  # Translate to Spanish

  Preparing metadata (setup.py) ... done
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached networkx-2.8.8-py3-none-any.whl.metadata (5.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
  Using cached torchaudio-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.7 MB

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
^C
